In [1]:
import os
import numpy as np
import pandas as pd
import cv2
import matplotlib.pyplot as plt
import csv
import seaborn as sns
from skimage.feature import hog
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from joblib import Parallel, delayed
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import (accuracy_score, classification_report, confusion_matrix, ConfusionMatrixDisplay)
import time
from sklearn.svm import SVC
from sklearn.preprocessing import StandardScaler
import torch
from sklearn.svm import SVC

In [2]:
class_names = {
    0: "Speed limit (20km/h)", 1: "Speed limit (30km/h)", 2: "Speed limit (50km/h)",
    3: "Speed limit (60km/h)", 4: "Speed limit (70km/h)", 5: "Speed limit (80km/h)",
    6: "End of speed limit (80km/h)", 7: "Speed limit (100km/h)", 8: "Speed limit (120km/h)",
    9: "No passing", 10: "No passing for vehicles over 3.5 metric tons",
    11: "Right-of-way at the next intersection", 12: "Priority road", 13: "Yield",
    14: "Stop", 15: "No vehicles", 16: "Vehicles over 3.5 metric tons prohibited",
    17: "No entry", 18: "General caution", 19: "Dangerous curve to the left",
    20: "Dangerous curve to the right", 21: "Double curve", 22: "Bumpy road",
    23: "Slippery road", 24: "Road narrows on the right", 25: "Road work",
    26: "Traffic signals", 27: "Pedestrians", 28: "Children crossing",
    29: "Bicycles crossing", 30: "Beware of ice/snow", 31: "Wild animals crossing",
    32: "End of all speed and passing limits", 33: "Turn right ahead",
    34: "Turn left ahead", 35: "Ahead only", 36: "Go straight or right",
    37: "Go straight or left", 38: "Keep right", 39: "Keep left",
    40: "Roundabout mandatory", 41: "End of no passing",
    42: "End of no passing for vehicles over 3.5 metric tons"
}

In [3]:
augmented_I_RGB = np.load('augmented_I_RGB.npy')      # Shape: (N, 32, 32, 3)
augmented_I_Gray = np.load('augmented_I_Gray.npy')    # Shape: (N, 32, 32)
augmented_L = np.load('augmented_L.npy')              # Shape: (N,)

#Verify shapes
print(f"RGB images: {augmented_I_RGB.shape}")
print(f"Grayscale images: {augmented_I_Gray.shape}")
print(f"Labels: {augmented_L.shape}")

RGB images: (78418, 32, 32, 3)
Grayscale images: (78418, 32, 32)
Labels: (78418,)


In [4]:
def extract_hog_features(image):
    return hog(image, pixels_per_cell=(4, 4), cells_per_block=(2, 2), visualize=False)

X = np.array(Parallel(n_jobs=-1)(delayed(extract_hog_features)(img) for img in augmented_I_Gray))
y = augmented_L  

In [5]:
X_train, X_test, y_train, y_test = train_test_split(X, y,test_size=0.2,random_state=42,stratify=y)

scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

In [10]:
#https://www.researchgate.net/publication/344399165_Traffic_Sign_Recognition_System_TSRS_SVM_and_Convolutional_Neural_Network
import os
os.environ['OMP_NUM_THREADS'] = '32'  # Use all 32 threads
os.environ['MKL_NUM_THREADS'] = '32'
os.environ['OPENBLAS_NUM_THREADS'] = '32'

svm = SVC(
    kernel='rbf',
    C=10,
    gamma='scale',
    class_weight='balanced',
    decision_function_shape='ovr',
    random_state=42,
    verbose=2,               
    cache_size=40000,        #40GB RAM cache
    max_iter=-1,             #No limit
    tol=1e-3,               #Slightly looser tolerance for speed
    shrinking=True          
)


print(f"Starting training on {len(X_train_scaled)} samples...")
start_time = time.time()
svm.fit(X_train_scaled, y_train)
training_time = time.time() - start_time

# Evaluate
y_pred = svm.predict(X_test_scaled)
accuracy = accuracy_score(y_test, y_pred)

print(f"\n=== Results ===")
print(f"Training time: {training_time/60:.1f} minutes")
print(f"Test accuracy: {accuracy:.4f}")
print(f"Support vectors: {len(svm.support_vectors_)}")

Starting training on 62734 samples...
[LibSVM]
=== Results ===
Training time: 53.9 minutes
Test accuracy: 0.9934
Support vectors: 37308


# Using GPU

In [9]:
from thundersvm import SVC  # Uses GPU instead of CPU

# Convert data to numpy (no need for PyTorch in this case)
X_train_np = X_train_scaled.astype('float32')
y_train_np = y_train.astype('int32')  # Ensure labels are integers

# Train SVM on GPU
svm = SVC(kernel='rbf', C=10, gamma='scale')
start_time = time.time()
svm.fit(X_train_np, y_train_np)  # Runs on GPU!
training_time = time.time() - start_time
print(f"Training time (GPU): {training_time:.2f}s")

FileNotFoundError: Please build the library first!

In [6]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Using device: {device}")

Using device: cuda
